# Specviz2D - Spectral Extraction

In [ ]:
import tempfile
import warnings
from astropy.utils.data import download_file
from astroquery.mast import Observations
from zipfile import ZipFile

from jdaviz import Mosviz, Specviz2d

warnings.simplefilter('ignore')

## Trace Selection

We'll use Mosviz to extract the 2D spectrum of a single object from the MOS data set shown in the Mosviz demo.

In [ ]:
# If downloading data from Box:
# example_data = 'https://stsci.box.com/shared/static/pg8f2vyb6lvn4flfetpetsprkh30e5ud.zip'
# data_dir = tempfile.gettempdir()
# fn = download_file(example_data, cache=False)
# with ZipFile(fn, 'r') as sample_data_zip:
#    sample_data_zip.extractall(data_dir)

# If running on JWebbinar platform:
data_dir = '/home/shared/preloaded-fits/jdaviz_data'

mosviz = Mosviz()
mosviz.load_data(directory=f"{data_dir}/115", instrument="niriss")

mosviz.app.get_viewer('table-viewer').select_row(1)
spectra2d = mosviz.app.get_data_from_viewer('spectrum-2d-viewer', 'F115W Source 219 spec2d C')

And then load that into Specviz2d.

In [ ]:
specviz2d = Specviz2d()
specviz2d.load_data(spectrum_2d=spectra2d)
specviz2d.show('sidecar:split-right')

The x-limits on both viewers are kept in sync, so if we set the zoom limits with any zoom tool (in the viewer toolbars) or through the API, the x-limits on the other viewer will update accordingly.

In [ ]:
specviz2d.specviz.x_limits(150, 215)
specviz2d.specviz.y_limits(0, 1e3)

Each plugin has its own API wrapper.  We can access these through the `specviz2d.plugins` dictionary.

In [ ]:
specviz2d.plugins

## Spectral Extraction

Calling `dir()` will show all available attributes and methods of the plugin API object.

In [ ]:
spext = specviz2d.plugins['Spectral Extraction']
dir(spext)

For more information on any of the methods, use `help(...)`  (`help(spext.open_in_tray)` or `spext.open_in_tray?`).

In [ ]:
help(spext.open_in_tray)

In [ ]:
spext.open_in_tray()

Now we can manually edit trace options or access/set them from the plugin API.

In [ ]:
print(spext.trace_pixel)

In [ ]:
spext.trace_pixel = 146
spext.bg_type = 'TwoSided'
spext.bg_separation = 60
spext.bg_width = 30

### Cycle through Rows

The API also allows for making calls to the plugin within more advanced scripts (including conditionals or for loops, etc).  Let's extract the spectrum for a single row at a time and cycle through the rows.  Here we want to disable background subtraction, by having the input to boxcar extraction be the 2D spectrum directly (instead of the background-subtracted image done on the fly within the plugin).  Every dropdown in the plugin has a corresponding attribute in the plugin object, including a list of the valid options:

In [ ]:
print(spext.ext_dataset)

In [ ]:
spext.ext_dataset = 'Spectrum 2D'

In [ ]:
for row in range(0, 250):
    spext.trace_pixel = row

## Trace Fitting

Now we'll start a new instance of `Specviz2d` and load a new data file that exhibits a non-linear spectrum.

In [ ]:
fn = 'jw01309-o022_s00003_nirspec_f290lp-g395h-s200a2-allslits_s2d.fits'

# If downloading from MAST:
# data_dir = tempfile.gettempdir()
# result = Observations.download_file(f"mast:JWST/product/{fn}", local_path=f'{data_dir}/{fn}')

# If running on JWebbinar platform:
data_dir = '/home/shared/preloaded-fits/jdaviz_data'

specviz2d = Specviz2d()
specviz2d.load_data(f'{data_dir}/{fn}')

specviz2d.show('sidecar:split-right')

In [ ]:
spext = specviz2d.plugins['Spectral Extraction']
spext.open_in_tray()

Now we can fit a polynomial trace with custom options, define a one-sided background, and use API calls to export the background image and extracted spectrum directly into the notebook.

Trace options (through UI or API):

In [ ]:
spext.trace_type = 'Polynomial'
spext.trace_order = 3
spext.trace_pixel = 23
spext.trace_bins = 10
spext.trace_window = 6

Background options (through UI or API):

In [ ]:
spext.bg_type = 'OneSided'
spext.bg_separation = 8
spext.bg_width = 8
spext.bg_statistic = 'Median'

In [ ]:
spext.export_bg_img()

Boxcar extraction options (through UI or API):

In [ ]:
spext.ext_width = 2
spext.export_extract_spectrum(add_data=True)